In [8]:
import datetime
import math
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import requests
import re
import tensorflow as tf

from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# import and load model architectures as well as decoder
from models.cueva import LSTM_FE
from models.llanes_jurado import LSTM_CNN
from utilities.preprocessors import correct_signals
from utilities.loaders import load_meta_data, load_model, load_lookup_array, save_meta_data, concur_load_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load engineered test features from data

In [9]:
models = {
    'cueva_second_phase-svm':{

    },
    'cueva-lstm-fe': {
        # 'model':
        # 'hyper_params':
    },
    'jurado-lstm-cnn': {
        # 'model':
        # 'hyper_params':
    },
    'taylor-svm': {
        # 'model':
        # 'selected_feats':
    },
    'taylor-lr': {
        # 'model':
        # 'selected_feats':
    },
    'taylor-rf': {
        # 'model':
        # 'selected_feats':
    },
    'hossain-gbt': {
        # 'model':
        # 'selected_feats':
        # 'scaler':
    },
    'hossain-svm': {
        # 'model':
        # 'selected_feats':
        # 'scaler':
    },
    'hossain-lr': {
        # 'model':
        # 'selected_feats':
        # 'scaler':
    }
}

In [10]:
def load_miscs():
    """
    loads miscellaneous variables to be used by the model
    """

    global models

    print('loading miscellaneous...')
    # this is for loading miscellaneous variables for 
    # deep learning models such as hyper parameters
    lstm_fe_hp = load_meta_data('./saved/misc/cueva_lstm-fe_meta_data.json')
    lstm_cnn_hp = load_meta_data('./saved/misc/jurado_lstm-cnn_meta_data.json')

    models['cueva-lstm-fe']['hyper_params'] = lstm_fe_hp
    models['jurado-lstm-cnn']['hyper_params'] = lstm_cnn_hp

    # this is for loading miscellaneous variables for
    # machine learning models such as the reduced feature set
    taylor_lr_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_taylor_lr_feature_set.txt')
    taylor_svm_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_taylor_svm_feature_set.txt')
    taylor_rf_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_taylor_rf_feature_set.txt')
    hossain_lr_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_hossain_lr_feature_set.txt')
    hossain_svm_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_hossain_svm_feature_set.txt')
    hossain_gbt_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_hossain_gbt_feature_set.txt')
    cueva_second_phase_svm_red_feats = load_lookup_array(f'./data/Artifact Detection Data/reduced_cueva_second_phase_svm_feature_set.txt')

    # pre-load reduced features here so that features don't have to 
    # be loaded every single time user makes a request
    models['taylor-lr']['selected_feats'] = taylor_lr_red_feats
    models['taylor-svm']['selected_feats'] = taylor_svm_red_feats
    models['taylor-rf']['selected_feats'] = taylor_rf_red_feats
    models['hossain-lr']['selected_feats'] = hossain_lr_red_feats
    models['hossain-svm']['selected_feats'] = hossain_svm_red_feats
    models['hossain-gbt']['selected_feats'] = hossain_gbt_red_feats
    models['cueva_second_phase-svm']['selected_feats'] = cueva_second_phase_svm_red_feats

    print('miscellaneous loaded.')


def load_preprocessors():
    """
    prepares and loads the saved encoders, normalizers of
    the dataset to later transform raw user input from
    client-side
    """
    global models

    print('loading preprocessors...')

    # pre-load here scaler of hossain used during training
    hossain_lr_scaler = load_model('./saved/misc/hossain_lr_scaler.pkl')
    hossain_svm_scaler = load_model('./saved/misc/hossain_svm_scaler.pkl')
    hossain_gbt_scaler = load_model('./saved/misc/hossain_gbt_scaler.pkl')

    models['hossain-lr']['scaler'] = hossain_lr_scaler
    models['hossain-svm']['scaler'] = hossain_svm_scaler
    models['hossain-gbt']['scaler'] = hossain_gbt_scaler

    print('preprocessors loaded.')

def load_models():
    """
    prepares and loads sample input and custom model in
    order to use trained weights/parameters/coefficients
    """
    global models
    
    print('loading models...')
    # pre load saved weights for deep learning models
    jurado_lstm_cnn = LSTM_CNN(**models['jurado-lstm-cnn']['hyper_params'])
    jurado_lstm_cnn.load_weights('./saved/weights/EDABE_LSTM_1DCNN_Model.h5')

    # load side task model and convert it to a feature extractor model 
    lstm_fe = LSTM_FE(**models['cueva-lstm-fe']['hyper_params'])
    lstm_fe.load_weights('./saved/weights/cueva_lstm-fe_21_0.7489.weights.h5')
    lstm_layer_2 = lstm_fe.get_layer('lstm-layer-2')
    lstm_fe_main = tf.keras.Model(inputs=lstm_fe.inputs, outputs=lstm_layer_2.output)

    # # pre load saved machine learning models
    taylor_lr = load_model('./saved/models/taylor_lr_clf.pkl')
    taylor_svm = load_model('./saved/models/taylor_svm_clf.pkl')
    taylor_rf = load_model('./saved/models/taylor_rf_clf.pkl')
    hossain_lr = load_model('./saved/models/hossain_lr_clf.pkl')
    # hossain_svm = load_model('./saved/models/hossain_svm_clf.pkl')
    hossain_gbt = load_model('./saved/models/hossain_gbt_clf.pkl')
    cueva_second_phase_svm = load_model('./saved/models/cueva_second_phase_svm_clf.pkl')

    # populate dictionary with loaded models
    models['jurado-lstm-cnn']['model'] = jurado_lstm_cnn
    models['cueva-lstm-fe']['model'] = lstm_fe

    models['taylor-lr']['model'] = taylor_lr
    models['taylor-svm']['model'] = taylor_svm
    models['taylor-rf']['model'] = taylor_rf
    models['hossain-lr']['model'] = hossain_lr
    # models['hossain-svm']['model'] = hossain_svm
    models['hossain-gbt']['model'] = hossain_gbt
    models['cueva_second_phase-svm']['model'] = cueva_second_phase_svm

    print('models loaded.')

In [11]:
load_miscs()
load_preprocessors()
load_models()

loading miscellaneous...
miscellaneous loaded.
loading preprocessors...
preprocessors loaded.
loading models...
models loaded.


In [12]:
models

{'cueva_second_phase-svm': {'selected_feats': ['HOF_2',
   'HOF_5',
   'HOF_7',
   'HOF_9',
   'HOF_10',
   'HOF_12',
   'HOF_13',
   'HOF_17',
   'HOF_18',
   'HOF_19',
   'HOF_20',
   'HOF_21',
   'HOF_22',
   'HOF_26',
   'HOF_32',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_min',
   'filt_128hz_1d_range',
   'filt_128hz_1d_max_abs',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'raw_128hz_skewness',
   'raw_128hz_kurt',
   'third_16thofa_sec_max',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_std',
   'first_16thofa_sec_median',
   'second_16thofa_sec_median',
   'third_16thofa_sec_range',
   'second_32thofa_sec_max',
   'first_32thofa_sec_mean',
   'second_32thofa_sec_mean',
   'first_32thofa_sec_std',
   'second_32thofa_sec_std',
   'first_32thofa_sec_median',
   'second_32thofa_sec_median',
   'first_32thofa_sec_range',
   'raw_16hz_min',
   'raw_16hz_range',
   

In [13]:
# model_names = ['cueva_second_phase-svm', 'taylor-lr', 'taylor-rf', 'taylor-svm', 'hossain-lr', 'hossain-gbt', 'hossain-svm']
model_names = ['cueva_second_phase-svm', 'taylor-lr', 'taylor-rf', 'taylor-svm', 'hossain-lr', 'hossain-gbt']

In [14]:
data_splits = ["test", "train"]

# Load engineered test and trained data

In [15]:
for model_name in model_names:
    selector_config, estimator_name = model_name.split('-', 1)
    
    for data_split in data_splits:
        loader_args = {
            'feat_config': selector_config, 
            'data_split': data_split,
            'exc_lof': False
        } if selector_config == 'cueva_second_phase' else {
            'feat_config': selector_config, 
            'data_split': data_split,
            'exc_lof': False
        }

        subjects_features, subjects_labels, subjects_names, subject_to_id = concur_load_data(**loader_args)
        print(f'selector config: {selector_config}')
        print(f'estimator name: {estimator_name}')
        # print(f'subjects features shape: {subjects_features.shape}')
        # print(f'subjects labels shape: {subjects_labels.shape}\n')
        
        # loop through each generated features dataframes from test subjects signals and feed repeatedly to a trained ml models
        for index, subject_name in enumerate(subjects_names):
            # print(f'subject features columns: {subjects_features[subjects_features['subject_id'] == index].columns}')
            print(f'subject: {subject_name}')
            # once features are extracted features selected during
            # tuning will be used in testing as done also during training

            selected_feats = models[model_name]['selected_feats']

            if loader_args.get('exc_lof') == None or loader_args.get('exc_lof') == False:
                subject_features = subjects_features.loc[subjects_features['subject_id'] == index, selected_feats]
                subject_labels = subjects_labels[subjects_labels['subject_id'] == index].drop(columns=['subject_id'])
                print(f'subject features shape: {subject_features.shape}')
                print(f'subject labels shape: {subject_labels.shape}\n')

            # this will only fire if exc_lof is not None or is false
            else:
                # if user excludes lower order features, higher order features will only be loaded
                subject_features = subjects_features[subjects_features['subject_id'] == index].drop(columns=['subject_id'])
                subject_labels = subjects_labels[subjects_labels['subject_id'] == index].drop(columns=['subject_id'])
                
                print(f'subject features shape: {subject_features.shape}')
                print(f'subject labels shape: {subject_labels.shape}\n')

            # convert features and labels into numpy matrices
            X = subject_features.to_numpy()
            Y = subject_labels.to_numpy().ravel()

            # if hossain is the researcher chosen the scaler used during training
            # will be used to scale the test subject features
            if selector_config == "hossain":    
                scaler = models[model_name]['scaler']
                X = scaler.transform(X)

            model = models[model_name]['model']
            Y_pred = model.predict(X)
            Y_pred_prob = model.predict_proba(X)
            print(f"predicted Y: {Y_pred}")
            print(f"unique values and counts: {np.unique(Y_pred, return_counts=True)}")
            print(f"true Y: {Y}")
            print(f"unique values and counts: {np.unique(Y, return_counts=True)}")

            # compute performance metric values for test subject
            acc = accuracy_score(y_true=Y, y_pred=Y_pred)
            prec = precision_score(y_true=Y, y_pred=Y_pred, average="weighted")
            rec = recall_score(y_true=Y, y_pred=Y_pred, average="weighted")
            f1 = f1_score(y_true=Y, y_pred=Y_pred, average="weighted")
            roc_auc = roc_auc_score(y_true=Y, y_score=Y_pred_prob[:, 1], average="weighted", multi_class="ovo")
            conf_matrix = confusion_matrix(Y, Y_pred).tolist()

            print(f"{data_split} acc: {acc} \
            \n{data_split} prec: {prec} \
            \n{data_split} rec: {rec} \
            \n{data_split} f1: {f1} \
            \n{data_split} roc_auc: {roc_auc} \
            \n{data_split} conf_matrix: {conf_matrix}")

            results = models[f'{model_name}'].get(f'{data_split}_results', [])
            results.append(
                (subject_name, {
                    f'{data_split}_acc': acc,
                    f'{data_split}_prec': prec, 
                    f'{data_split}_rec': rec,
                    f'{data_split}_f1': f1,
                    f'{data_split}_roc_auc': roc_auc,
                    f'{data_split}_conf_matrix': conf_matrix
                })
            )
            models[f'{model_name}'][f'{data_split}_results'] = results

subjects features, labels, names and subject to id lookup loaded
selector config: cueva_second_phase
estimator name: svm
subject: ajeric_expert2
subject features shape: (12761, 60)
subject labels shape: (12761, 1)

predicted Y: [0 0 0 ... 0 0 0]
unique values and counts: (array([0, 1], dtype=int64), array([10498,  2263], dtype=int64))
true Y: [0 0 0 ... 0 0 0]
unique values and counts: (array([0, 1], dtype=int64), array([11360,  1401], dtype=int64))
test acc: 0.8672517827756445             
test prec: 0.9027227672609223             
test rec: 0.8672517827756445             
test f1: 0.8802495355228019             
test roc_auc: 0.871081395582632             
test conf_matrix: [[10082, 1278], [416, 985]]
subject: ukudab_expert2
subject features shape: (14287, 60)
subject labels shape: (14287, 1)

predicted Y: [0 0 0 ... 1 1 1]
unique values and counts: (array([0, 1], dtype=int64), array([10944,  3343], dtype=int64))
true Y: [0 0 0 ... 1 1 1]
unique values and counts: (array([0, 1], dtyp

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.6s


predicted Y: [0. 0. 0. ... 1. 0. 0.]
unique values and counts: (array([0., 1.]), array([11782,   989], dtype=int64))
true Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([11370,  1401], dtype=int64))
test acc: 0.9187221047686164             
test prec: 0.9104973720924875             
test rec: 0.9187221047686164             
test f1: 0.9124397502412224             
test roc_auc: 0.8825474893231813             
test conf_matrix: [[11057, 313], [725, 676]]
subject: ukudab_expert2
subject features shape: (14297, 40)
subject labels shape: (14297, 1)



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.8s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.8s


predicted Y: [0. 0. 0. ... 1. 1. 0.]
unique values and counts: (array([0., 1.]), array([12877,  1420], dtype=int64))
true Y: [0. 0. 0. ... 1. 1. 0.]
unique values and counts: (array([0., 1.]), array([12713,  1584], dtype=int64))
test acc: 0.8894873050290271             
test prec: 0.8845447771025483             
test rec: 0.8894873050290271             
test f1: 0.886824771216531             
test roc_auc: 0.8121278564771448             
test conf_matrix: [[12005, 708], [872, 712]]
subject: afegip_expert1
subject features shape: (12528, 40)
subject labels shape: (12528, 1)



...

predicted Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([10116,  2290], dtype=int64))
true Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([10116,  2290], dtype=int64))
train acc: 1.0             
train prec: 1.0             
train rec: 1.0             
train f1: 1.0             
train roc_auc: 1.0             
train conf_matrix: [[10116, 0], [0, 2290]]
subject: ihikay_expert1
subject features shape: (13826, 40)
subject labels shape: (13826, 1)



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.9s


predicted Y: [0. 0. 0. ... 1. 0. 0.]
unique values and counts: (array([0., 1.]), array([7512, 6314], dtype=int64))
true Y: [0. 0. 0. ... 1. 0. 0.]
unique values and counts: (array([0., 1.]), array([7512, 6314], dtype=int64))
train acc: 1.0             
train prec: 1.0             
train rec: 1.0             
train f1: 1.0             
train roc_auc: 1.0             
train conf_matrix: [[7512, 0], [0, 6314]]
subject: asifex_expert2
subject features shape: (11437, 40)
subject labels shape: (11437, 1)



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.6s


predicted Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([11000,   437], dtype=int64))
true Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([11000,   437], dtype=int64))
train acc: 1.0             
train prec: 1.0             
train rec: 1.0             
train f1: 1.0             
train roc_auc: 0.9999999999999999             
train conf_matrix: [[11000, 0], [0, 437]]
subjects features, labels, names and subject to id lookup loaded
selector config: taylor
estimator name: svm
subject: ajeric_expert2
subject features shape: (12771, 40)
subject labels shape: (12771, 1)

predicted Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([12478,   293], dtype=int64))
true Y: [0. 0. 0. ... 0. 0. 0.]
unique values and counts: (array([0., 1.]), array([11370,  1401], dtype=int64))
test acc: 0.9047842768773001             
test prec: 0.896873894048558             
test rec: 0.9047842768773001             
test f1: 0.8

In [16]:
models

{'cueva_second_phase-svm': {'selected_feats': ['HOF_2',
   'HOF_5',
   'HOF_7',
   'HOF_9',
   'HOF_10',
   'HOF_12',
   'HOF_13',
   'HOF_17',
   'HOF_18',
   'HOF_19',
   'HOF_20',
   'HOF_21',
   'HOF_22',
   'HOF_26',
   'HOF_32',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_min',
   'filt_128hz_1d_range',
   'filt_128hz_1d_max_abs',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'raw_128hz_skewness',
   'raw_128hz_kurt',
   'third_16thofa_sec_max',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_std',
   'first_16thofa_sec_median',
   'second_16thofa_sec_median',
   'third_16thofa_sec_range',
   'second_32thofa_sec_max',
   'first_32thofa_sec_mean',
   'second_32thofa_sec_mean',
   'first_32thofa_sec_std',
   'second_32thofa_sec_std',
   'first_32thofa_sec_median',
   'second_32thofa_sec_median',
   'first_32thofa_sec_range',
   'raw_16hz_min',
   'raw_16hz_range',
   

In [17]:
for model_name in ["jurado-lstm-cnn"]:
    selector_config, estimator_name = model_name.split('-', 1)

    for data_split in data_splits:
        subjects_signals, subjects_labels, subjects_names, subject_to_id = concur_load_data(feat_config=selector_config, data_split=data_split)
        print(f'selector config: {selector_config}')
        print(f'estimator name: {estimator_name}')

        for index, subject_name in enumerate(subjects_names):
            X = subjects_signals[index]
            Y = subjects_labels[index]

            # assign model of jurado to variable
            model = models[model_name]['model']

            # depending on dl model Y_pred will either be unactivated logits 
            # or sigmoid probabilities
            Y_pred = model.predict(X)
            
            # when our predictions is 0.2, 0.15, and below which is <= 0.2 then y_pred will be 0
            # when our predictions is 1, 0.5, 0.4, 0.3, 0.21, and above which is > 0.2 then y_pred will be 1
            # why we do this is because of the imbalance of our dataset, and we
            # want to place a threshold of 20% since there our dataset only consists
            # of 20% of positive classes. Note this conversion is to be used in precision and recall metrics
            Y_pred_whole = tf.cast(Y_pred >= 0.2, tf.int64)

            print(f"predicted Y: {Y_pred_whole}")
            print(f"unique values and counts: {np.unique(Y_pred_whole, return_counts=True)}")
            print(f"true Y: {Y}")
            print(f"unique values and counts: {np.unique(Y, return_counts=True)}")

            # compute performance metric values for test subject
            acc = accuracy_score(y_true=Y, y_pred=Y_pred_whole)
            prec = precision_score(y_true=Y, y_pred=Y_pred_whole, average="weighted")
            rec = recall_score(y_true=Y, y_pred=Y_pred_whole, average="weighted")
            f1 = f1_score(y_true=Y, y_pred=Y_pred_whole, average="weighted")
            roc_auc = roc_auc_score(y_true=Y, y_score=Y_pred, average="weighted", multi_class="ovo")
            conf_matrix = confusion_matrix(Y, Y_pred_whole).tolist()

            print(f"{data_split} acc: {acc} \
                \n{data_split} prec: {prec} \
                \n{data_split} rec: {rec} \
                \n{data_split} f1: {f1} \
                \n{data_split} roc_auc: {roc_auc} \
                \n{data_split} conf_matrix: {conf_matrix}")
            
            results = models[model_name].get(f'{data_split}_results', [])
            results.append(
                (subject_name, {
                    f'{data_split}_acc': acc,
                    f'{data_split}_prec': prec, 
                    f'{data_split}_rec': rec,
                    f'{data_split}_f1': f1,
                    f'{data_split}_roc_auc': roc_auc,
                    f'{data_split}_conf_matrix': conf_matrix
                })
            )
            models[model_name][f'{data_split}_results'] = results

length of x_signals: 770814
window size: 640
length of x_signals: 817326
window size: 640
length of x_signals: 765045
window size: 640
length of x_signals: 762960
window size: 640
length of x_signals: 856086
window size: 640
length of x_signals: 801735
window size: 640
length of x_signals: 840582
window size: 640
length of x_signals: 871590
window size: 640
length of x_signals: 914989
window size: 640
length of x_signals: 980118
window size: 640
number of rows created: 12034
number of rows created: 12761
number of rows created: 11944
number of rows created: 11912
number of rows created: 12518
number of rows created: 13609
number of rows created: 13125
number of rows created: 13367
number of rows created: 14287
number of rows created: 15305
subjects signals, labels, names and subject to id lookup loaded
selector config: jurado
estimator name: lstm-cnn
399/399 ━━━━━━━━━━━━━━━━━━━━ 32s 78ms/step
predicted Y: [[1]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
unique values and counts: (array([0, 1], dtyp

In [18]:
models

{'cueva_second_phase-svm': {'selected_feats': ['HOF_2',
   'HOF_5',
   'HOF_7',
   'HOF_9',
   'HOF_10',
   'HOF_12',
   'HOF_13',
   'HOF_17',
   'HOF_18',
   'HOF_19',
   'HOF_20',
   'HOF_21',
   'HOF_22',
   'HOF_26',
   'HOF_32',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_min',
   'filt_128hz_1d_range',
   'filt_128hz_1d_max_abs',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'raw_128hz_skewness',
   'raw_128hz_kurt',
   'third_16thofa_sec_max',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_std',
   'first_16thofa_sec_median',
   'second_16thofa_sec_median',
   'third_16thofa_sec_range',
   'second_32thofa_sec_max',
   'first_32thofa_sec_mean',
   'second_32thofa_sec_mean',
   'first_32thofa_sec_std',
   'second_32thofa_sec_std',
   'first_32thofa_sec_median',
   'second_32thofa_sec_median',
   'first_32thofa_sec_range',
   'raw_16hz_min',
   'raw_16hz_range',
   

In [19]:
copy = models.copy()

In [20]:
for key, value in copy.items():
    if copy[key].get('model') is not None:
        del copy[key]['model']

    if copy[key].get('scaler') is not None:
        del copy[key]['scaler']

In [21]:
copy

{'cueva_second_phase-svm': {'selected_feats': ['HOF_2',
   'HOF_5',
   'HOF_7',
   'HOF_9',
   'HOF_10',
   'HOF_12',
   'HOF_13',
   'HOF_17',
   'HOF_18',
   'HOF_19',
   'HOF_20',
   'HOF_21',
   'HOF_22',
   'HOF_26',
   'HOF_32',
   'raw_128hz_std',
   'raw_128hz_range',
   'raw_128hz_shannon_entropy',
   'raw_128hz_1d_shannon_entropy',
   'filt_128hz_min',
   'filt_128hz_1d_range',
   'filt_128hz_1d_max_abs',
   'ar_coeff_1_128hz',
   'ar_coeff_2_128hz',
   'raw_128hz_skewness',
   'raw_128hz_kurt',
   'third_16thofa_sec_max',
   'first_16thofa_sec_mean',
   'second_16thofa_sec_mean',
   'third_16thofa_sec_std',
   'first_16thofa_sec_median',
   'second_16thofa_sec_median',
   'third_16thofa_sec_range',
   'second_32thofa_sec_max',
   'first_32thofa_sec_mean',
   'second_32thofa_sec_mean',
   'first_32thofa_sec_std',
   'second_32thofa_sec_std',
   'first_32thofa_sec_median',
   'second_32thofa_sec_median',
   'first_32thofa_sec_range',
   'raw_16hz_min',
   'raw_16hz_range',
   

In [22]:
save_meta_data('./results/all_models_results.json', copy)